<img style="float: left;" src="figures/model3.png" width="20%">   

# <font color='Red'>   CCS in depleted field </font>

## <font color='Blue'>Introduction</font>

In this notebook, we work on an example of CO2 sequestration in a depleted gas reservoir. Here again, we will be using predefined Python model from teh following files:
 * File [radial near-well model](https://gitlab.com/open-darts/darts-models/-/blob/development/teaching/CCS_workshop/depleted/nearwellbore.py) with structure and unstructured discretizers and plotters 
 * File [model](https://gitlab.com/open-darts/darts-models/-/blob/development/teaching/CCS_workshop/depleted/model.py) defining reservoir, physics, wells and output


To start import all the necessary Python modules and files.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from darts.engines import value_vector, redirect_darts_output
from depleted.model import Model

redirect_darts_output('model3.log')

## <font color='Blue'>Reservoir parameters</font>

Here we use a radial near-well model and define our reservoir.

In [ ]:
# define the model
m = Model()

(nr, nz) = (100, 24)
if 1:
    case = 'depleted_gas_field'
    poro = np.ones((nr, nz)) * 0.2
    perm = np.ones((nr, nz)) * 20

else:
    case = 'Porthos'
    poro = np.ones((nr, nz)) * 0.075
    perm = np.ones((nr, nz)) * 0.29
    # upper detfurth
    perm[:, :16] = 12.6
    # hardegsen
    poro[:, :10] = 0.09
    perm[:, :10] = 24
    # hardegsen high perm
    poro[:, :8] = 0.2
    perm[:, :8] = 240
    # hardegsen
    poro[:, :6] = 0.09
    perm[:, :6] = 24
    # caprock
    poro[:, :4] = 0.01
    perm[:, :4] = 0.01

m.set_reservoir_radial(nr=nr, dr=5, nz=nz, dz=5, poro=poro.flatten(order='F'), perm=perm.flatten(order='F'))

## <font color='Blue'>Initialize the model</font>

In [ ]:
# define physics
zero = 1e-12
m.swc = 0.25
m.set_physics(zero, n_points=1001, components=["CO2", "C1", "H2O"])

# simulation parameters
m.set_sim_params(first_ts=1e-4, mult_ts=4, max_ts=365, tol_newton=1e-3, tol_linear=1e-6, 
                 it_newton=16, it_linear=50)

# injection conditions
m.inj_stream = [1 - 2 * zero, zero, 320]
m.p_inj = 60.
m.p_init = 40.

# find saturation corresponding with composition
z_range = np.linspace(zero, 1 - zero, 10000)
for z in z_range:
    state = [m.p_init, zero, z, 370]
    sat = m.physics.property_containers[0].compute_saturation_full(state)
    if m.physics.property_containers[0].sat[m.physics.phases.index("Aq")] < m.swc:
        break

# initial conditions
m.initial_values = {"pressure": state[0],
                    "CO2": state[1],
                    "C1": state[2],
                    "temperature": state[3],
                    }

# initialization of model
m.init()
m.set_output(output_folder = 'output/' + case)

## <font color='Blue'>Run the model</font>

We run the model for 5 years and output solution every year.

In [ ]:
output_props = ['satV', 'pressure', 'temperature']
m.output_to_vtk(ith_step = 0, output_properties = output_props) # initial conditions 

for i in range(5):
    m.run(365)
    m.output_to_vtk(ith_step = i+1, output_properties = output_props) # save a .vtk every year

# print timers and statistics for the run
m.print_timers()
m.print_stat()

## <font color='Blue'>3D reservoir plot</font>

In [ ]:
import pyvista as pv
import os

# get vts data
sol = pv.read(os.path.join(m.output_folder, 'vtk_files', 'solution5.vtk'))

# define plotter
plotter = pv.Plotter()

# set temperature as active scalar
sol.set_active_scalars('temperature')

# add threshold levels
thresT = sol.clip('z', invert=True, )

# add outline of mesh
outline = sol.outline()

# add elements to plotter
plotter.set_background('#52576c')
plotter.add_mesh(outline, color = 'k')
plotter.add_mesh(thresT, cmap = 'coolwarm', show_edges=False)

plotter.camera_position = [-3,-5,2] 
plotter.camera.zoom(1.3)
plotter.show()

## <font color='Blue'>Tasks in this workshop (check and explain why the solution behaves this way):</font>

1. Check what happends if initial $P$ is 3 times higher keeping the same $\Delta P$.
2. Return original initial $P$, check what happend if injection $T$ is equal to initial $T$.
3. For initial $P$ and $T$ conditions, check the effect of heterogeneity on reservoir cooling.

More details on the the modeling depleted fields can be found in https://doi.org/10.2118/223902-MS.